In [19]:
import requests
import json
import pandas as pd

# Список ISIN кодов облигаций, которые нас интересуют
TARGET_ISIN_LIST = [
    "SU26238RMFS4", "SU26233RMFS5", "SU26240RMFS0", "SU26218RMFS6",
    "SU26230RMFS1", "SU26225RMFS1", "RU000A106HB4", "RU000A104JQ3",
    "RU000A107MM9", "RU000A102T63", "RU000A103PX8", "SU26207RMFS9",
    "SU26219RMFS4", "SU26226RMFS9", "SU26229RMFS3", "SU26232RMFS7",
    "SU26243RMFS4", "SU29014RMFS6", "SU29016RMFS1", "SU29022RMFS9",
    "SU29025RMFS2", "RU000A106K43", "RU000A103D37", "RU000A106516"
]

# Преобразуем в set для быстрого поиска
TARGET_ISIN_SET = set(TARGET_ISIN_LIST)

# Базовый URL API Московской биржи (ISS MOEX)
BASE_URL = "https://iss.moex.com/iss"

def fetch_target_securities():
    """
    Загружает информацию о бумагах из TARGET_ISIN_SET.
    Возвращает словарь {isin: {details...}}
    """
    securities_data = {}
    isin_query = ",".join(TARGET_ISIN_SET)
    url = f"{BASE_URL}/engines/stock/markets/bonds/securities.json?securities={isin_query}&iss.meta=off"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if 'securities' in data and 'columns' in data['securities'] and 'data' in data['securities']:
            cols = data['securities']['columns']
            rows = data['securities']['data']
            
            # Находим индексы нужных колонок
            try:
                isin_idx = cols.index('SECID')  # ISIN часто в SECID
                name_idx = cols.index('SECNAME')
                couponvalue_idx = cols.index('COUPONVALUE')  # Размер купона
                couponperiod_idx = cols.index('COUPONPERIOD') # Период купона
                matdate_idx = cols.index('MATDATE') # Дата погашения
                nextcoupon_idx = cols.index('NEXTCOUPON') # Дата следующего купона
                # >>> ДОБАВЬТЕ НОВЫЕ ПОЛЯ ЗДЕСЬ <<<
                # Например, если хотите добавить поле "MATDATE" (дата погашения):
                # matdate_idx = cols.index('MATDATE')
                
            except ValueError as e:
                print(f"Ошибка: Не найдена необходимая колонка ({e}).")
                return {}  # Возвращаем пустой словарь при ошибке структуры
            
            for row in rows:
                isin = row[isin_idx]
                if isin in TARGET_ISIN_SET:
                    securities_data[isin] = {
                        'isin': isin,
                        'name': row[name_idx],
                        'coupon_value': float(row[couponvalue_idx]) if row[couponvalue_idx] else 0.0,
                        'coupon_period': float(row[couponperiod_idx]) if row[couponperiod_idx] else 0.0,
                        'mat_date': row[matdate_idx],
                        'next_coupon': row[nextcoupon_idx]
                        # >>> ДОБАВЬТЕ НОВЫЕ ПОЛЯ ЗДЕСЬ <<<
                        # Например, если добавили "MATDATE":
                        # 'maturity_date': row[matdate_idx],
                    }
        else:
            print("Блок 'securities' не найден в ответе или пуст.")
    except requests.exceptions.RequestException as e:
        print(f"Ошибка сети при загрузке данных: {e}")
    except json.JSONDecodeError:
        print("Ошибка декодирования JSON.")
    except Exception as e:
        print(f"Неизвестная ошибка: {e}")
    
    return securities_data

# --- Основная часть скрипта ---
print("Этап 1: Загрузка информации о целевых облигациях...")
all_securities_info = fetch_target_securities()
print(f"-> Найдено {len(all_securities_info)} ISIN из целевого списка.")

print("-" * 60)
print("Этап 2: Создание DataFrame...")

# Готовим структуру DataFrame
dataframe_rows = []
for isin, details in all_securities_info.items():
    row_data = {
        'Название': details.get('name', 'N/A'),
        'Код (ISIN)': details.get('isin', 'N/A'),
        'Размер купона': details.get('coupon_value', 0.0),
        'Период купона': details.get('coupon_period', 0.0),
        'Дата погашения': details.get('mat_date', 0.0),
        'Дата следующего купона': details.get('next_coupon', 0.0),
        # >>> ДОБАВЬТЕ НОВЫЕ ПОЛЯ ЗДЕСЬ <<<
        # Например, если добавили "MATDATE":
        # 'Дата погашения': details.get('maturity_date', 'N/A'),
    }
    dataframe_rows.append(row_data)

# Создаем DataFrame
if dataframe_rows:
    df = pd.DataFrame(dataframe_rows)
else:
    df = pd.DataFrame(columns=['Название', 'Код (ISIN)', 'Размер купона'])
# Преобразуем столбец 'Дата следующего купона' в формат datetime
df['Дата следующего купона'] = pd.to_datetime(df['Дата следующего купона'])
# Создаем новый столбец с порядковым номером месяца
df['Номер месяца'] = df['Дата следующего купона'].dt.month
df["Сумма купонов за год"] = round (365 / df["Период купона"],0)*df['Размер купона']
print("-" * 60)
print("Итоговый DataFrame:")
print(df)
print("-" * 60)
print("Завершено.")

Этап 1: Загрузка информации о целевых облигациях...
-> Найдено 23 ISIN из целевого списка.
------------------------------------------------------------
Этап 2: Создание DataFrame...
------------------------------------------------------------
Итоговый DataFrame:
                     Название    Код (ISIN)  Размер купона  Период купона  \
0       ОФЗ-ПД 26207 03/02/27  SU26207RMFS9          40.64          182.0   
1       ОФЗ-ПД 26218 17/09/31  SU26218RMFS6          42.38          182.0   
2       ОФЗ-ПД 26219 16/09/26  SU26219RMFS4          38.64          182.0   
3       ОФЗ-ПД 26225 10/05/34  SU26225RMFS1          36.15          182.0   
4       ОФЗ-ПД 26226 07/10/26  SU26226RMFS9          39.64          182.0   
5       ОФЗ-ПД 26229 12/11/25  SU26229RMFS3          35.65          182.0   
6       ОФЗ-ПД 26232 06/10/27  SU26232RMFS7          29.92          182.0   
7     ОФЗ-ПД 26233 18/07/2035  SU26233RMFS5          30.42          182.0   
8     ОФЗ-ПД 26240 30/07/2036  SU26240RMFS0 